In [26]:
!pip install langchain_experimental

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 18.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 14.3 MB/s eta 0:00:0000:01


In [40]:
from langchain_experimental.tools.python.tool import PythonAstREPLTool
import pandas as pd

df = pd.read_csv("data/titanic.csv")
python_repl_ast = PythonAstREPLTool(locals={'df':df})
tool_map = {'python_repl_ast': python_repl_ast}

tool = 'python_repl_ast'
tool_input = 'df.head(3)'

tool_map[tool].run(tool_input)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Unnamed: 12
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,NaN
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN


In [55]:
from utils import chat_invoke
from utils import template_substitute

input = '年龄大于22岁的乘客共几名'
scratchpad = [
        {
            "Thought":"回答用户问题，前我需要观察一下数据例名",
            "Tool": "python_repl_ast",
            "Tool Input":"df.columns",
            "Observation":"Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Unnamed: 12'], dtype='object')"
        }
    ]

prompt = template_substitute('agent', input=input, scratchpad=scratchpad)
result = chat_invoke(prompt)
print(result)

 
```json
{
    "Question": "年龄大于22岁的乘客共几名？",
    "Scratchpad": [
        {
            "Thought": "回答这个问题，我需要计算年龄大于22岁的乘客数量。可以通过筛选年龄列中大于22的值来实现。",
            "Tool": "python_repl_ast",
            "Tool Input": "df[df['Age'] > 22].shape[0]"
        }
    ],
    "Thought": "我已经编写了相应的代码来计算年龄大于22岁的乘客数量。",
    "Tool": "python_repl_ast",
    "Tool Input": "df[df['Age'] > 22].shape[0]",
    "Observation": null,
    "Finished": 0
}
```


In [42]:
from utils import json_loads

obj = json_loads(result)
thought  = obj['Thought']
tool = obj['Tool']
tool_input = obj['Tool Input']
observation = tool_map[tool].run(tool_input)
print(observation)

483



In [56]:
scratchpad.append({
    'Thought':thought,
    'Tool': tool,
    'Tool Input':tool_input,
    'observation':observation
})

scratchpad

[{'Thought': '回答用户问题，前我需要观察一下数据例名',
  'Tool': 'python_repl_ast',
  'Tool Input': 'df.columns',
  'Observation': "Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Unnamed: 12'], dtype='object')"},
 {'Thought': "要回答这个问题，我需要计算年龄大于22岁的乘客数量。这可以通过筛选df中'Age'列大于22的行来实现。",
  'Tool': 'python_repl_ast',
  'Tool Input': "print(df[df['Age'] > 22].shape[0])",
  'observation': '483\n'}]

In [57]:
prompt = template_substitute('agent', input=input, scratchpad=scratchpad)
result = chat_invoke(prompt)
print(result)

 
```json
{
    "Question": "年龄大于22岁的乘客共几名？",
    "Scratchpad": [
        {
            "Thought": "要回答这个问题，我需要计算年龄大于22岁的乘客数量。这可以通过筛选df中'Age'列大于22的行来实现。",
            "Tool": "python_repl_ast",
            "Tool Input": "print(df[df['Age'] > 22].shape[0])",
            "Observation": "None"
        }
    ],
    "Thought": "我已经写好了计算年龄大于22岁的乘客数量的代码，接下来我将执行这段代码。",
    "Tool": "python_repl_ast",
    "Tool Input": "print(df[df['Age'] > 22].shape[0])",
    "Observation": "483",
    "Finished": 1,
    "Answer": "年龄大于22岁的乘客共有483名。"
}
```


In [67]:
scratchpad = [
        {
            "Thought":"回答用户问题，前我需要观察一下数据例名",
            "Tool": "python_repl_ast",
            "Tool Input":"df.columns",
            "Observation":"Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Unnamed: 12'], dtype='object')"
        }
    ]


def agent(query, scratchpad={}, max_request_time=10):
    cur_request_imte = 0
    finished = 0
    while cur_request_imte < max_request_time and finished == 0:
        cur_request_imte += 1
        obj = invoke(query, scratchpad)
        
        if obj['Finished'] == 1:
            finished = 1
            print('Final Answer: ', obj['Answer'])
        else:
            finished == obj['Finished']
            scratchpad = excute(obj, scratchpad)
                        
def excute(obj, scratchpad):
    # 执行代码获得结果
    tool = obj['Tool']
    tool_input = obj['Tool Input']
    observation = tool_map[tool].run(tool_input) 
    
    # 记录scatchpad
    thought = obj['Thought']
    scratchpad.append({
        'Thought':thought,
        'Tool': tool,
        'Tool Input':tool_input,
        'observation':observation
    })
    
    return scratchpad

def invoke(query, scratchpad):
    prompt = template_substitute('agent', input=query, scratchpad=scratchpad)
    # print('INPUT'.center(30, '#'), prompt, sep='\n')
    result = chat_invoke(prompt)
    # print('OUTPUT'.center(30, '#'), result, sep='\n')
    return json_loads(result)

In [68]:
agent('30岁以上女性有几名', scratchpad)

Final Answer:  30岁以上的女性乘客有103名。
